## Nature of Pixar Films Project

#### Parts to project:
1. Scrape Pandas DF from the-numbers
2. Store + Copy info from columns of DF
3. For each non-null movie (movies that haven't been released yet), scrape reviews from RottenTomatoes.com 
4. Find reviews with words that hint towards environment or gender roles
5. Sum count and List out

In [200]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
import html5lib
import unicodedata as ucd
import html.parser
import numpy as np
import contractions as cont
import nltk
import re
import tensorflow
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, Embedding, LSTM
from keras.utils import to_categorical
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from tqdm import tqdm_notebook
tokenizer = ToktokTokenizer()

lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('wordnet')
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ashleychu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ashleychu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [201]:
train = pd.read_csv("train.tsv", sep="\t")
test = pd.read_csv("test.tsv", sep="\t")
train.head()
train.shape

(156060, 4)

In [202]:
#Note: 61 pages of movie statistics
def getMovieStatistics():
    """
    TODO: Get box-office and budget data from the-numbers.com
    OUT: A Pandas Dataframe with the following columns:
    Title  |  Year  |  Budget  |  Domestic Gross  |  International Gross
    """
    
    link = "https://www.the-numbers.com/movies/production-company/Pixar"
    r = rq.get(link)
    tb = pd.read_html(r.text)
    return pd.concat(tb)
allMovieStats = getMovieStatistics()
allMovieStats = allMovieStats.dropna()
allMovieStats

ReleaseDate                Title ProductionBudget OpeningWeekend  \
2   Jun 21, 2019          Toy Story 4     $200,000,000   $120,908,065   
3   Jun 15, 2018        Incredibles 2     $200,000,000   $182,687,905   
4   Nov 22, 2017                 Coco     $175,000,000    $50,802,605   
5   Jun 16, 2017               Cars 3     $175,000,000    $53,688,680   
6   Jun 17, 2016         Finding Dory     $200,000,000   $135,060,273   
7   Nov 25, 2015    The Good Dinosaur     $187,500,000    $39,155,217   
8   Jun 19, 2015           Inside Out     $175,000,000    $90,440,272   
9   Jun 21, 2013  Monsters University     $200,000,000    $82,429,469   
10  Jun 22, 2012                Brave     $185,000,000    $66,323,594   
11  Jun 24, 2011               Cars 2     $200,000,000    $66,135,507   
12  Jun 18, 2010          Toy Story 3     $200,000,000   $110,307,189   
14  May 29, 2009                   Up     $175,000,000    $68,108,790   
15  Jun 27, 2008               WALL-E     $180,000,000    $63,087,526   
16  Jun 29, 2007          Ratatouille     $150,000,000    $47,027,395   
17   Jun 9, 2006                 Cars      $70,000,000    $60,119,509   
18   Nov 5, 2004      The Incredibles      $92,000,000    $70,467,623   
19  May 30, 2003         Finding Nemo      $94,000,000    $70,251,710   
20   Nov 2, 2001       Monsters, Inc.     $115,000,000    $62,577,067   
21  Nov 19, 1999          Toy Story 2      $90,000,000       $300,163   
22  Nov 20, 1998         A Bug's Life      $45,000,000       $291,121   
23  Nov 22, 1995            Toy Story      $30,000,000    $29,140,617   

   DomesticBox Office WorldwideBox Office  
2        $434,038,008      $1,073,394,813  
3        $608,581,744      $1,242,805,359  
4        $209,726,015        $799,976,015  
5        $152,901,115        $383,541,369  
6        $486,295,561      $1,028,570,889  
7        $123,087,120        $333,771,037  
8        $356,461,711        $857,561,711  
9        $268,488,329        $743,588,329  
10       $237,282,182        $554,606,532  
11       $191,450,875        $560,155,383  
12       $415,004,880      $1,068,879,522  
14       $293,004,164        $731,463,377  
15       $223,808,164        $532,508,025  
16       $206,445,654        $626,549,695  
17       $244,082,982        $461,651,246  
18       $261,441,092        $631,441,092  
19       $380,529,370        $936,429,370  
20       $289,423,425        $559,757,719  
21       $245,852,179        $511,358,276  
22       $162,798,565        $363,095,319  
23       $191,796,233        $364,545,516

In [232]:
years = list(allMovieStats['ReleaseDate'])
movie_yr = []
for year in years:
    movie_yr.append(year.split(' ')[2])

titles = list(allMovieStats['Title'])
sumBudgets = np.array(list(allMovieStats['WorldwideBox Office']))
for i in range(len(sumBudgets)):
    item = sumBudgets[i]
    item = item.replace('$','')
    item = item.replace(',', '')
    sumBudgets[i] = int(item)
    
holder = np.array(list(allMovieStats['ProductionBudget']))
for i in range(len(holder)):
    item = holder[i]
    item = item.replace('$','')
    item = item.replace(',', '')
    holder[i] = int(item)

total = []
for i in range(len(holder)):
    if (holder[i] == 'nan'):
        total.append(0)
    else:
        total.append(int(sumBudgets[i]) - int(holder[i]))

df['Year'] = movie_yr
df['Movie'] = titles
df['Revenue'] = total
df['Revenue'] = df['Revenue'].map('${:,.2f}'.format)
df

Movie  Year            Revenue  \
0           Toy Story 4  2019    $873,394,813.00   
1         Incredibles 2  2018  $1,042,805,359.00   
2                  Coco  2017    $624,976,015.00   
3                Cars 3  2017    $208,541,369.00   
4          Finding Dory  2016    $828,570,889.00   
5     The Good Dinosaur  2015    $146,271,037.00   
6            Inside Out  2015    $682,561,711.00   
7   Monsters University  2013    $543,588,329.00   
8                 Brave  2012    $369,606,532.00   
9                Cars 2  2011    $360,155,383.00   
10          Toy Story 3  2010    $868,879,522.00   
11                   Up  2009    $556,463,377.00   
12               WALL-E  2008    $352,508,025.00   
13          Ratatouille  2007    $476,549,695.00   
14                 Cars  2006    $391,651,246.00   
15      The Incredibles  2004    $539,441,092.00   
16         Finding Nemo  2003    $842,429,370.00   
17       Monsters, Inc.  2001    $444,757,719.00   
18          Toy Story 2  1999    $421,358,276.00   
19         A Bug's Life  1998    $318,095,319.00   
20            Toy Story  1995    $334,545,516.00   

                                              Reviews  Word Count  
0                                                  []           0  
1   [While original, witty and super entertaining,...          19  
2                                                  []           0  
3   [Better than the second film but not as good a...          24  
4   [The dip back into the flow of some of Pixar's...          34  
5   [What rises the entertainment quality of The G...          33  
6   [Great filmmaking is empathetic. It lets us ex...          76  
7                                                  []           0  
8   [Brave is a physically gorgeous work that succ...          26  
9   [Yes, the movie has some weak spots. A little ...          33  
10  [While threepeats may be rare in the sports wo...          48  
11  [Yet, with the story perfectly setup for a cla...          15  
12  [Ultimately, WALL-E is about nothing less than...          52  
13  [The studio's latest effort offers beautiful a...          52  
14  [Pixar's grown so metallic that having automob...          11  
15                                                 []           0  
16  [This gem of a film is the one with which Pixa...          70  
17  [It's funny, the characters are endearing, the...          30  
18  [Toy Story 2 shows how pop art can disarm your...          18  
19  [The rest of the film has similar fun mixing b...          10  
20  [Toy Story remains the humanistic, vital film ...          20

In [244]:
def getReviews(movie, year):
    """
    TODO: return an array of the rottentomatoes critic reviews given the movie and the year it was released
    IN: movie title(string) and year(string)
    OUT: array of reviews(strings)
    """
    
    link_format = "https://rottentomatoes.com/m/"
    movie_title = ""
    count = len(movie.split(' '))
    for word in movie.split(' '):
        if (":" in word):
            word = word.replace(":", "")
        elif ("Ep." in word):
            word = word.replace("Ep.", "episode")
        elif ("-" in word):
            word = word.replace("-", "_")
        elif("'" in word):
            word = word.replace("'", "")
        elif("." in word):
            word = word.replace(".", "")
        elif("," in word):
            word = word.replace(",", "")
        movie_title += word
        if(count > 1):
            movie_title += "_"
        count -= 1;
    
    if (movie == "A Bug's Life"):
        movie_title = "bugs_life"
    elif (movie == "The Incredibles"):
        movie_title = "incredibles"
        
    link = link_format + movie_title + "_" + year + "/reviews"
    html = rq.get(link)

    if(not html.ok):
        link = link_format + movie_title + "/reviews"
        html = rq.get(link)
    if(not html.ok):
        return None

    print(link)
    
    htmlSoup = BeautifulSoup(html.text)
    
    if htmlSoup.find('span', {'class':'pageInfo'}) is None:
        return None
    
    
    amount_of_pages_arr = (htmlSoup.find('span', {'class':'pageInfo'})).text.split(' ')
    page_count = int(amount_of_pages_arr[len(amount_of_pages_arr)-1])
    count = 0
    reviews = []
#     genderCt = []
    
#     wordToCheck = ["environment", "climate", "diversity", "biodiversity", "sentiment", "sentiments", "human", "connection", "realistic", "real-world", "issues", "world", "ecology", "ecosystem", "capitalism", "conservation", "disposable", "plastic", "waste", "deforestation", "future", "global", "pollution", "conserve", "conservation", "eco", "eco-friendly", "ecological"]
    wordToCheck = ["diversity", "female", "women", "stigma", "caretaker", "empowerment", "gender", "roles", "society", "non-binary", "binary", "LGBTQ+", "pride", "sex"]
    
    for i in range(1, page_count + 1):
        curr_link = link + "?type=&sort=&page=" + str(i)
        html_per_rev_get = rq.get(curr_link)
        html_per_rev = BeautifulSoup(html_per_rev_get.text)
        item = htmlSoup.find('div', {'class' : 'the_review'})
        while item:
            textToAdd = item.text.strip()
            if any(word in textToAdd.lower() for word in wordToCheck):
                reviews.append(textToAdd)
#             if any(word in textToAdd.lower() for word in genderWords):
#                 reviews.append(textToAdd)
            item = item.findNext('div', {'class' : 'the_review'})
            count += 1

    return count

In [234]:
totalSum = []
for i in range(len(titles)):
    t = titles[i]
    yr = movie_yr[i]
    totalSum.append(getReviews(t, yr))

https://rottentomatoes.com/m/Toy_Story_4/reviews
https://rottentomatoes.com/m/Incredibles_2/reviews
https://rottentomatoes.com/m/Coco_2017/reviews
https://rottentomatoes.com/m/Cars_3_2017/reviews
https://rottentomatoes.com/m/Finding_Dory_2016/reviews
https://rottentomatoes.com/m/The_Good_Dinosaur_2015/reviews
https://rottentomatoes.com/m/Inside_Out_2015/reviews
https://rottentomatoes.com/m/Monsters_University/reviews
https://rottentomatoes.com/m/Brave_2012/reviews
https://rottentomatoes.com/m/Cars_2/reviews
https://rottentomatoes.com/m/Toy_Story_3/reviews
https://rottentomatoes.com/m/Up/reviews
https://rottentomatoes.com/m/WALL_E/reviews
https://rottentomatoes.com/m/Ratatouille/reviews
https://rottentomatoes.com/m/Cars/reviews
https://rottentomatoes.com/m/incredibles/reviews
https://rottentomatoes.com/m/Finding_Nemo/reviews
https://rottentomatoes.com/m/Monsters_Inc/reviews
https://rottentomatoes.com/m/Toy_Story_2/reviews
https://rottentomatoes.com/m/bugs_life/reviews
https://rottentoma

In [235]:
df['Env Reviews'] = totalSum
# df['G Reviews'] = sumTwo
allNums = []
# allNums2 = []
for arr in totalSum:
    if (arr is None):
        allNums.append(None)
    else:
        allNums.append(len(arr))
# for arr in sumTwo:
#     if (arr is None):
#         allNums2.append(None)
#     else:
#         allNums2.append(len(arr))

df['Env Count'] = allNums
# df['Gender Count'] = allNums2

In [238]:

df.sort_values(['Env Count'], ascending=False)

Movie  Year            Revenue  \
6            Inside Out  2015    $682,561,711.00   
16         Finding Nemo  2003    $842,429,370.00   
12               WALL-E  2008    $352,508,025.00   
13          Ratatouille  2007    $476,549,695.00   
10          Toy Story 3  2010    $868,879,522.00   
4          Finding Dory  2016    $828,570,889.00   
5     The Good Dinosaur  2015    $146,271,037.00   
9                Cars 2  2011    $360,155,383.00   
17       Monsters, Inc.  2001    $444,757,719.00   
8                 Brave  2012    $369,606,532.00   
3                Cars 3  2017    $208,541,369.00   
20            Toy Story  1995    $334,545,516.00   
1         Incredibles 2  2018  $1,042,805,359.00   
18          Toy Story 2  1999    $421,358,276.00   
11                   Up  2009    $556,463,377.00   
14                 Cars  2006    $391,651,246.00   
19         A Bug's Life  1998    $318,095,319.00   
15      The Incredibles  2004    $539,441,092.00   
7   Monsters University  2013    $543,588,329.00   
2                  Coco  2017    $624,976,015.00   
0           Toy Story 4  2019    $873,394,813.00   

                                          Env Reviews  Env Count  
6   [Great filmmaking is empathetic. It lets us ex...         76  
16  [This gem of a film is the one with which Pixa...         70  
12  [Ultimately, WALL-E is about nothing less than...         52  
13  [The studio's latest effort offers beautiful a...         52  
10  [While threepeats may be rare in the sports wo...         48  
4   [The dip back into the flow of some of Pixar's...         34  
5   [What rises the entertainment quality of The G...         33  
9   [Yes, the movie has some weak spots. A little ...         33  
17  [It's funny, the characters are endearing, the...         30  
8   [Brave is a physically gorgeous work that succ...         26  
3   [Better than the second film but not as good a...         24  
20  [Toy Story remains the humanistic, vital film ...         20  
1   [While original, witty and super entertaining,...         19  
18  [Toy Story 2 shows how pop art can disarm your...         18  
11  [Yet, with the story perfectly setup for a cla...         15  
14  [Pixar's grown so metallic that having automob...         11  
19  [The rest of the film has similar fun mixing b...         10  
15                                                 []          0  
7                                                  []          0  
2                                                  []          0  
0                                                  []          0

In [241]:
totalSum = []
for i in range(len(titles)):
    t = titles[i]
    yr = movie_yr[i]
    totalSum.append(getReviews(t, yr))
    
df['Gen Reviews'] = totalSum
allNums = []
for arr in totalSum:
    if (arr is None):
        allNums.append(None)
    else:
        allNums.append(len(arr))

df['Gen Count'] = allNums

https://rottentomatoes.com/m/Toy_Story_4/reviews
https://rottentomatoes.com/m/Incredibles_2/reviews
https://rottentomatoes.com/m/Coco_2017/reviews
https://rottentomatoes.com/m/Cars_3_2017/reviews
https://rottentomatoes.com/m/Finding_Dory_2016/reviews
https://rottentomatoes.com/m/The_Good_Dinosaur_2015/reviews
https://rottentomatoes.com/m/Inside_Out_2015/reviews
https://rottentomatoes.com/m/Monsters_University/reviews
https://rottentomatoes.com/m/Brave_2012/reviews
https://rottentomatoes.com/m/Cars_2/reviews
https://rottentomatoes.com/m/Toy_Story_3/reviews
https://rottentomatoes.com/m/Up/reviews
https://rottentomatoes.com/m/WALL_E/reviews
https://rottentomatoes.com/m/Ratatouille/reviews
https://rottentomatoes.com/m/Cars/reviews
https://rottentomatoes.com/m/incredibles/reviews
https://rottentomatoes.com/m/Finding_Nemo/reviews
https://rottentomatoes.com/m/Monsters_Inc/reviews
https://rottentomatoes.com/m/Toy_Story_2/reviews
https://rottentomatoes.com/m/bugs_life/reviews
https://rottentoma

In [243]:
df.sort_values(['Env Count'], ascending=False)

Movie  Year            Revenue  \
6            Inside Out  2015    $682,561,711.00   
16         Finding Nemo  2003    $842,429,370.00   
12               WALL-E  2008    $352,508,025.00   
13          Ratatouille  2007    $476,549,695.00   
10          Toy Story 3  2010    $868,879,522.00   
4          Finding Dory  2016    $828,570,889.00   
5     The Good Dinosaur  2015    $146,271,037.00   
9                Cars 2  2011    $360,155,383.00   
17       Monsters, Inc.  2001    $444,757,719.00   
8                 Brave  2012    $369,606,532.00   
3                Cars 3  2017    $208,541,369.00   
20            Toy Story  1995    $334,545,516.00   
1         Incredibles 2  2018  $1,042,805,359.00   
18          Toy Story 2  1999    $421,358,276.00   
11                   Up  2009    $556,463,377.00   
14                 Cars  2006    $391,651,246.00   
19         A Bug's Life  1998    $318,095,319.00   
15      The Incredibles  2004    $539,441,092.00   
7   Monsters University  2013    $543,588,329.00   
2                  Coco  2017    $624,976,015.00   
0           Toy Story 4  2019    $873,394,813.00   

                                          Env Reviews  Env Count  \
6   [Great filmmaking is empathetic. It lets us ex...         76   
16  [This gem of a film is the one with which Pixa...         70   
12  [Ultimately, WALL-E is about nothing less than...         52   
13  [The studio's latest effort offers beautiful a...         52   
10  [While threepeats may be rare in the sports wo...         48   
4   [The dip back into the flow of some of Pixar's...         34   
5   [What rises the entertainment quality of The G...         33   
9   [Yes, the movie has some weak spots. A little ...         33   
17  [It's funny, the characters are endearing, the...         30   
8   [Brave is a physically gorgeous work that succ...         26   
3   [Better than the second film but not as good a...         24   
20  [Toy Story remains the humanistic, vital film ...         20   
1   [While original, witty and super entertaining,...         19   
18  [Toy Story 2 shows how pop art can disarm your...         18   
11  [Yet, with the story perfectly setup for a cla...         15   
14  [Pixar's grown so metallic that having automob...         11   
19  [The rest of the film has similar fun mixing b...         10   
15                                                 []          0   
7                                                  []          0   
2                                                  []          0   
0                                                  []          0   

                                          Gen Reviews  Gen Count  
6                                                  []          0  
16                                                 []          0  
12                                                 []          0  
13                                                 []          0  
10                                                 []          0  
4                                                  []          0  
5                                                  []          0  
9                                                  []          0  
17                                                 []          0  
8   [...call me picky, but it seems a shame that a...         26  
3   [Overall I really like Cars 3 a ton. It has a ...         12  
20                                                 []          0  
1                                                  []          0  
18                                                 []          0  
11                                                 []          0  
14                                                 []          0  
19                                                 []          0  
15  [The Incredibles is a very sophisticated movie...         13  
7                                                  []          0  
2                                   

In [237]:
# *use normalize functionus
# convert from unicode to ascii
# encode and decode -> encode -> allows putting it into ascii -> decode allows to take it out of UTF-8


def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser") 
#     transfer = unicodedata.normalize('NFKD', text)
    return soup.get_text()
    
    
def remove_accented_chars(text):
    text = ucd.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text
    
def expand_contractions(text):
#     cont.contract_texts(text)
    return cont.fix(text)
    
    
def remove_special_chars(text):
    pattern = r'[^a-zA-Z/s]'
    return re.sub(pattern, '', text)
    
print(remove_special_chars('#cars'))

# simplifies text
def lemmatize_text(text):
    return lemmatizer.lemmatize(text)

def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

def remove_digits(text):
    return False
    

cars
